In [79]:
#import the required libraries
import cv2
import mediapipe as mp
import numpy as np
import time
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose
# Function to calculate angle
def calculate_Angle(a,b,c):
    a=np.array(a) #first
    b=np.array(b) #mid
    c=np.array(c)#end
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180.0:
        angle=360.0-angle
    return angle

# Initialize MediaPipe Pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize video capture
cap = cv2.VideoCapture('D:/AI Assignment-Knee movemnet.mp4')  # Replace with your video file path

# Variables for curl counter
counter = 0
stage = None
holding_start_time = None
holding_threshold = 5  # Holding time threshold in seconds

while cap.isOpened():
    ret, frame = cap.read()
    try:
        # Convert the image to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
    
        # Process pose detection
        result = pose.process(image_rgb)
    
        image_rgb.flags.writeable = True
    
        # Convert the image back to BGR
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
   
    
        # Extract landmarks
       
        landmarks = result.pose_landmarks.landmark
    
        # Get coordinates
        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y]
        knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y]
        ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y]
        #calculate the angle 
        angle = calculate_Angle(hip, knee, ankle)
        
        text_position = (int(1280 * 0.03), int(720 * 0.1))
        # Visualize angle
        cv2.putText(image_bgr, "Angle:{:.2f}".format(angle),
                text_position,
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
        #visualize count
        cv2.putText(image_bgr, "Count:{}".format(counter),
                (int(1280 * 0.03), int(720 * 0.2)),  # Adjust position as needed
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
        
        if angle < 49 and stage == 'down':
            stage = 'up'
            counter += 1
            print(counter)
            
        if angle > 50:
            stage = 'down'
            feedback_message = "Hold the knee in the bent position for 5 sec"
            cv2.putText(image_bgr, str(feedback_message),
                        (int(1280 * 0.03), int(720 * 0.3)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
            holding_triggered = True
            holding_start_time = time.time()

        # holding timer logic
        if holding_triggered and angle < 49:
            holding_elapsed_time = time.time() - holding_start_time
            if holding_elapsed_time > holding_threshold:
                print("Holding time exceeded. Resetting holding timer.")
                holding_triggered = False
                holding_start_time = 0
            else:
                cv2.putText(image_bgr, f"Holding time: {holding_elapsed_time:.2f} seconds",
                        (int(1280 * 0.03), int(720 * 0.4)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
    except:
        pass

    # Draw landmarks on the image
    mp_drawing.draw_landmarks(image_bgr, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow('Knee exercise', image_bgr)
    # Break the loop and release resources on 'q
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#thanks for this project 

1
2
3
4
